In [1]:
%matplotlib inline
import networkx as nx
import sys
import math
import random
from random import randint
from collections import defaultdict, deque
print(nx.__version__)

2.0


In [2]:
# Disable plot axes by default
import matplotlib as mpl
if True:
    mpl.rc('axes.spines', top=False, bottom=False, left=False, right=False)
    mpl.rc('xtick', top=False, bottom=False, labelsize=0)
    mpl.rc('ytick', left=False, right=False, labelsize=0)

In [3]:
from IPython.display import Image, HTML

def pydot_image(graph, prog='dot', width=None):
    dot = nx.nx_pydot.to_pydot(graph)
    if width:
        return Image(dot.create_png(prog=prog), width=width)
    else:
        return HTML(dot.create_svg(prog=prog).decode())

# Минимальное вершинное покрытие в двудольном графе.

На вход подается описание двудольного графа, в котором доли уже выделены явно. Первая строка содержит три натуральных числа: v1<100v1<100 — число вершин первой доли, v2<100v2<100 — число вершин второй доли, e≤v1∗v2e≤v1∗v2 — число рёбер. Подразумевается, что первая доля состоит из вершин с номерами от 00 до v1−1v1−1, вторая — из вершин с номерами от v1v1 до v1+v2−1v1+v2−1. Следующие ee строк описывают рёбра: каждая из этих строк содержит два числа: 0≤ui<v10≤ui<v1 и v1≤wi<v1+v2v1≤wi<v1+v2, что означает, что между вершинами uiui и wiwi есть ребро.

Скопируйте описание графа из входа на выход и выведите единственную дополнительную строку — список номеров вершин, составляющих минимальное вершинное покрытие. Если таких покрытий несколько, выведите любое.

In [4]:
test_input = "2 2 3\n0 2\n0 3\n1 3\n"

In [5]:
def random_bipartite(max_left=10, max_right=20, min_edges=10, max_edges=100):
    from networkx import Graph, is_connected, relabel_nodes
    from networkx.algorithms.bipartite.generators import gnmk_random_graph
    from networkx.algorithms.bipartite import sets as bipartite_sets
    from random import randint
    while True:
        n = randint(1, max_left)
        m = randint(1, max_right)
        k = randint(min(m,n), max(m,n,max_edges))
        g0 = gnmk_random_graph(n,m,k)
        g = Graph()
        g.add_nodes_from(set(g0.nodes))
        edges = set()
        for a,b in g0.edges:
            if a != b:
                a,b = min(a,b), max(a,b)
                edges.add((a,b))
        g.add_edges_from(edges)
        if is_connected(g):
            break
    left, right = bipartite_sets(g)
    n_left, n_right = len(left), len(right)
    mapping = {v:i for i,v in enumerate(sorted(left))}
    for i,v in enumerate(sorted(right)):
        mapping[v] = i + n_left
    g = relabel_nodes(g, mapping, copy=True)
    header = '%d %d %d' % (n_left, n_right, len(g.edges))
    edges = '\n'.join('%d %d' % (min(a,b),max(a,b)) for a,b in sorted(g.edges))
    return '%s\n%s\n' % (header, edges)

In [6]:
import random
random.seed(8035336)
all_tests = [random_bipartite() for _ in range(500)]

In [7]:
def to_seq(input_data):
    if isinstance(input_data, str):
        return (line for line in input_data.splitlines())
    else:
        return input_data

def nx_min_cover(input_data):
    from networkx import Graph
    from networkx.algorithms.bipartite import maximum_matching, to_vertex_cover
    input_seq = to_seq(input_data)
    nleft, nright, nedges = map(int, next(input_seq).split())
    nverts = nleft + nright
    g = Graph()
    g.add_nodes_from(range(nverts))
    edges = set()
    for _ in range(nedges):
        a,b = map(int, next(input_seq).split())
        a,b = min(a,b), max(a,b)
        edges.add((a,b))
    g.add_edges_from(edges)
    matching = maximum_matching(g)
    cover = to_vertex_cover(g, matching)
    return ' '.join(map(str, sorted(cover)))

print(nx_min_cover(test_input))

0 1


In [8]:
import sys
from collections import defaultdict, deque
from itertools import cycle, chain

def ok_min_cover(input_data, full=False):
    INF = float('inf')

    if isinstance(input_data, str):
        input_seq = (line for line in input_data.splitlines())
    elif input_data == sys.stdin:
        def input_gen():
            while 1:
                yield input()
        input_seq = input_gen()
    else:
        input_seq = input_data

    line = next(input_seq)
    out_lines = [line]
    nleft, nright, nedges = map(int, line.split())
    nverts = nleft + nright

    adj = defaultdict(set)
    edges = set()
    for i in range(nedges):
        line = next(input_seq)
        out_lines.append(line)
        a,b = map(int, line.split())
        a,b = min(a,b),max(a,b)
        if a == b:
            continue
        adj[a].add(b)
        edges.add((a,b))
        adj[b].add(a)

    nodes = set(range(nverts))
    left = set(range(0, nleft))
    right = set(range(nleft, nverts))

    def bfs():
        for v in left:
            if mleft[v] is None:
                dist[v] = 0
                queue.append(v)
            else:
                dist[v] = INF
        dist[None] = INF
        while queue:
            v = queue.popleft()
            if dist[v] < dist[None]:
                for u in adj[v]:
                    if dist[mright[u]] is INF:
                        dist[mright[u]] = dist[v] + 1
                        queue.append(mright[u])
        return dist[None] is not INF

    def dfs(v):
        if v is not None:
            for u in adj[v]:
                if dist[mright[u]] == dist[v] + 1:
                    if dfs(mright[u]):
                        mright[u] = v
                        mleft[v] = u
                        return True
            dist[v] = INF
            return False
        return True

    mleft = {v: None for v in left}
    mright = {v: None for v in right}
    dist = {}
    queue = deque()

    nmatched = 0
    while bfs():
        for v in left:
            if mleft[v] is None:
                if dfs(v):
                    nmatched += 1

    mleft = {k: v for k, v in mleft.items() if v is not None}
    mright = {k: v for k, v in mright.items() if v is not None}

    matching = dict(chain(mleft.items(), mright.items()))

    unmatched_vertices = nodes - set(matching)
    targets = unmatched_vertices & left
    
    edge_sets = {frozenset((u, v)) for u, v in matching.items()}
    matched_edges = {tuple(edge) for edge in edge_sets}
    unmatched_edges = {(u, v) for (u, v) in edges if frozenset((u, v)) not in edge_sets}

    def _alt_dfs(u, for_matched):
        edges = (cycle([matched_edges, unmatched_edges])
                 if for_matched else
                 cycle([unmatched_edges, matched_edges]))
        visited = set()
        stack = [(u, iter(adj[u]), next(edges))]
        while stack:
            parent, children, valid_edges = stack[-1]
            try:
                child = next(children)
                if child not in visited:
                    if ((parent, child) in valid_edges or (child, parent) in valid_edges):
                        if child in targets:
                            return True
                        visited.add(child)
                        stack.append((child, iter(adj[child]), next(edges)))
            except StopIteration:
                stack.pop()
        return False

    zset = {v for v in nodes if v in targets or _alt_dfs(v, True) or _alt_dfs(v, False)}
    result =  (left - zset) | (right & zset)

    out = ' '.join(map(str, sorted(result)))
    out_lines.append(out)
    if full:
        out = '\n'.join(out_lines)
    return out

print(ok_min_cover(test_input))
#print(ok_min_cover(sys.stdin, full=True))
for graph in all_tests:
    nxr = nx_min_cover(graph)
    okr = ok_min_cover(graph)
    if nxr != okr:
        print('fail', repr(graph))
print('done')

0 1
fail '6 6 17\n0 6\n0 7\n0 8\n0 9\n0 10\n0 11\n1 7\n1 8\n1 9\n1 11\n2 6\n2 7\n3 6\n3 9\n4 7\n5 6\n5 9\n'
fail '9 8 32\n0 15\n0 16\n1 9\n1 12\n1 13\n1 15\n2 9\n2 10\n2 12\n2 13\n2 15\n2 16\n3 10\n3 11\n4 9\n4 13\n4 14\n4 16\n5 11\n5 13\n5 14\n5 16\n6 9\n6 10\n6 11\n6 13\n6 16\n7 9\n7 11\n7 12\n7 16\n8 12\n'
done


In [9]:
# troubleshoot